In [33]:
import os

# Define the folder number
folder_number = '13'  # Change this to the specific folder number you are working with

# Define the paths to the JSON files
base_path = f'results_Jul23/slowjam/{folder_number}/'
channels_json_path = os.path.join(base_path, 'channels.json')
forwarding_history_json_path = os.path.join(base_path, 'forwarding_history.json')


In [34]:
import pandas as pd
import json

# Load channels.json
with open(channels_json_path, 'r') as file:
    channels_data = json.load(file)

# Load forwarding_history.json
with open(forwarding_history_json_path, 'r') as file:
    forwarding_history_data = json.load(file)

# Extract the forwarding events
forwards = forwarding_history_data.get('forwards', [])

# Create the initial DataFrame
rows = []
for entry in forwards:
    if isinstance(entry, dict):
        rows.append({
            'addTimeNs': int(entry['addTimeNs']),
            'resolveTimeNs': int(entry.get('resolveTimeNs', 0)),
            'eventTimeNs': int(entry['addTimeNs']),
            'eventType': 'add',
            'incomingAmount': int(entry['incomingAmount']),
            'outgoingAmount': int(entry['outgoingAmount']),
            'shortChannelId_outgoing': int(entry['outgoingCircuit']['shortChannelId']),
            'shortChannelId_incoming': int(entry['incomingCircuit']['shortChannelId']),
        })
        if 'resolveTimeNs' in entry:
            rows.append({
                'addTimeNs': int(entry['addTimeNs']),
                'resolveTimeNs': int(entry['resolveTimeNs']),
                'eventTimeNs': int(entry['resolveTimeNs']),
                'eventType': 'resolve',
                'incomingAmount': int(entry['incomingAmount']),
                'outgoingAmount': int(entry['outgoingAmount']),
                'shortChannelId_outgoing': int(entry['outgoingCircuit']['shortChannelId']),
                'shortChannelId_incoming': int(entry['incomingCircuit']['shortChannelId']),
            })

pair_schedule_df = pd.DataFrame(rows)

# Sort the DataFrame by eventTimeNs
pair_schedule_df = pair_schedule_df.sort_values(by='eventTimeNs').reset_index(drop=True)

In [35]:
# Create a DataFrame for channel capacities
channels_df = pd.DataFrame(channels_data['channels'])
channels_df['chan_id'] = channels_df['chan_id'].astype(int)
channels_df['capacity'] = channels_df['capacity'].astype(int)
channels_df = channels_df[['chan_id', 'capacity']]

In [55]:
def track_funds(pair_schedule_df, channels_df):
    # Initialize a dictionary to track available funds
    funds = {channel: {'incoming': cap // 2, 'outgoing': cap // 2} for channel, cap in zip(channels_df['chan_id'], channels_df['capacity'])}
    
    # List to store times when locked funds exceed 80%
    high_locked_times = []
    status = []
    for _, row in pair_schedule_df.iterrows():
        if (row['shortChannelId_incoming'] not in {545357767442432, 551954837209088}) and (row['shortChannelId_outgoing'] not in {545357767442432, 551954837209088}):
            if row['eventType'] == 'add':
                funds[row['shortChannelId_incoming']]['outgoing'] += row['incomingAmount']
                funds[row['shortChannelId_outgoing']]['incoming'] += row['outgoingAmount']
            elif row['eventType'] == 'resolve':
                funds[row['shortChannelId_incoming']]['outgoing'] -= row['incomingAmount']
                funds[row['shortChannelId_outgoing']]['incoming'] -= row['outgoingAmount']


            # Check if the locked amounts exceed 80% of available funds
            for channel, cap in zip(channels_df['chan_id'], channels_df['capacity']):
                status.append({
                        'time': row['eventTimeNs'],
                        'channel': channel,
                        'capacity': cap,
                        'incoming_locked': funds[channel]['incoming'],
                        'outgoing_locked': funds[channel]['outgoing']
                    })
                if funds[channel]['incoming'] < 0.2 * (cap // 2) or funds[channel]['outgoing'] < 0.2 * (cap // 2):
                    high_locked_times.append({
                        'time': row['eventTimeNs'],
                        'channel': channel,
                        'capacity': cap,
                        'incoming_locked': funds[channel]['incoming'],
                        'outgoing_locked': funds[channel]['outgoing']
                    })

    return pd.DataFrame(high_locked_times), pd.DataFrame(status)

high_locked_df, st_df = track_funds(pair_schedule_df, channels_df)


In [58]:
pair_schedule_df.head(10)

,addTimeNs,resolveTimeNs,eventTimeNs,eventType,incomingAmount,outgoingAmount,shortChannelId_outgoing,shortChannelId_incoming
0,1721147252481120296,1721147257687071296,1721147252481120296,add,1053,53,393625162809344,378232000020480
1,1721147252481120296,1721147257687071296,1721147257687071296,resolve,1053,53,393625162809344,378232000020480
2,1721147277182630296,1721147279530726296,1721147277182630296,add,1053,53,393625162809344,378232000020480
3,1721147277182630296,1721147279530726296,1721147279530726296,resolve,1053,53,393625162809344,378232000020480
4,1721147307126044296,1721147310025065296,1721147307126044296,add,1053,53,393625162809344,378232000020480
5,1721147307126044296,1721147310025065296,1721147310025065296,resolve,1053,53,393625162809344,378232000020480
6,1721147331139158296,1721147339232994296,1721147331139158296,add,1053,53,393625162809344,378232000020480
7,1721147332300443296,1721147339235121296,1721147332300443296,add,1002,2,393625162809344,378232000020480
8,1721147331139158296,1721147339232994296,1721147339232994296,resolve,1053,53,393625162809344,378232000020480
9,1721147332300443296,1721147339235121296,1721147339235121296,resolve,1002,2,393625162809344,378232000020480


In [57]:
st_df.head(50)

,time,channel,capacity,incoming_locked,outgoing_locked
0,1721147252481120296,367236883742720,84803717,42401858,42401858
1,1721147252481120296,378232000020480,1000000,500000,501053
2,1721147252481120296,393625162809344,100000000,50000053,50000000
3,1721147252481120296,361739325603840,3000000,1500000,1500000
4,1721147252481120296,379331511648256,2000000,1000000,1000000
5,1721147252481120296,404620279087104,25000000,12500000,12500000
6,1721147252481120296,346346162814976,1620000,810000,810000
7,1721147257687071296,367236883742720,84803717,42401858,42401858
8,1721147257687071296,378232000020480,1000000,500000,500000
9,1721147257687071296,393625162809344,100000000,50000000,50000000


In [51]:
high_locked_df

,time,channel,capacity,incoming_locked,outgoing_locked
0,1721147495211192296,379331511648256,2000000,1000000,-10280974
1,1721147762387478296,379331511648256,2000000,1000000,-10280974
2,1721147767220015296,379331511648256,2000000,1000000,-10280974
3,1721147805439112296,379331511648256,2000000,1000000,-10616578
4,1721147805439112296,346346162814976,1620000,-10804998,810000
...,...,...,...,...,...
216,1721162769861213296,379331511648256,2000000,1000000,-10616578
217,1721162769861213296,346346162814976,1620000,-10804998,810000
218,1721162954594054296,379331511648256,2000000,1000000,-10280974
219,1721162955740366296,379331511648256,2000000,1000000,-10280974


In [47]:
funds = {channel: {'incoming': cap // 2, 'outgoing': cap // 2} for channel, cap in zip(channels_df['chan_id'], channels_df['capacity'])}


In [48]:
funds

{367236883742720: {'incoming': 42401858, 'outgoing': 42401858},
 378232000020480: {'incoming': 500000, 'outgoing': 500000},
 393625162809344: {'incoming': 50000000, 'outgoing': 50000000},
 361739325603840: {'incoming': 1500000, 'outgoing': 1500000},
 379331511648256: {'incoming': 1000000, 'outgoing': 1000000},
 404620279087104: {'incoming': 12500000, 'outgoing': 12500000},
 346346162814976: {'incoming': 810000, 'outgoing': 810000}}